**Aluno**: Lucas Peres Gaspar

**Matrícula**: 409504

**Nível**: Mestrando

**Programa**: Mestrado e Doutorado em Ciência da Computação

---

O objetivo deste trabalho é analisar um conjunto de dados sobre crédito bancário a fim de reconhecer padrões dentre os usuários e prever o status do pagamento. O código foi desenvolvido em Python 3 utilizando as bibliotecas Numpy, Pandas e o Scikit-Learn(a fins de otimização, uma vez que o dataset é grande e uma implementação própria seria computacionalmente custosa), bem como o ambiente de programação Jupyter Notebook. Este trabalho encontra-se no [GitHub](https://github.com/lucaspg96/pattern-recognition/tree/work2/work2), assim como os códigos-fonte.

Primeiramente, devemos importar as bibliotecas que serão utilizadas durante as análises.

In [1]:
import sys
sys.path.append('../')
import numpy as np
import pandas as pd

from algorithms.utils import train_test_split

from algorithms.ConfusionMatrix import ConfusionMatrix

from algorithms.Gaussian import QuadraticGaussianClassifier, NormalNaiveBayes
from algorithms.NearestNeighbors import NearestCentroidClassifier
from algorithms.Quadratic import QuadraticClassifier

from sklearn.neighbors import KNeighborsClassifier

import logging

logging.basicConfig(format='%(asctime)s - %(name)s - %(levelname)s - %(message)s')
logger = logging.getLogger()
logger.setLevel(logging.DEBUG)

Utilizamos o Pandas para visualizar os dados de maneira tabular, a fim de identificar como os dados estão organizados.

In [2]:
df = pd.read_csv("../datasets/default of credit card clients.csv", delimiter=',')
df.head()

,ID,X1,X2,X3,X4,X5,X6,X7,X8,X9,...,X15,X16,X17,X18,X19,X20,X21,X22,X23,Y
0,1,20000,2,2,1,24,2,2,-1,-1,...,0,0,0,0,689,0,0,0,0,1
1,2,120000,2,2,2,26,-1,2,0,0,...,3272,3455,3261,0,1000,1000,1000,0,2000,1
2,3,90000,2,2,2,34,0,0,0,0,...,14331,14948,15549,1518,1500,1000,1000,1000,5000,0
3,4,50000,2,2,1,37,0,0,0,0,...,28314,28959,29547,2000,2019,1200,1100,1069,1000,0
4,5,50000,1,2,1,57,-1,0,-1,0,...,20940,19146,19131,2000,36681,10000,9000,689,679,0


Podemos, também, fazer uma contagem da ocorrência das classes

In [3]:
df['Y'].value_counts()

0    23364
1     6636
Name: Y, dtype: int64

Temos 30000 observações para utilizarmos em nossos algoritmos. Isso torna custoso, tanto em tempo quanto em recurso computacional, a execução dos algoritmos de classificação. Além disso, podemos notar um desbalanceamento entre as classes.

## Classificação dos dados
---

Primeiramente, vamos executar os algoritmos de classificação sobre os dados originais e computar as métricas para avaliá-los.

In [4]:
data = np.loadtxt("../datasets/default of credit card clients.csv", delimiter=',', skiprows=1)
data.shape

(30000, 25)

Utilizaremos os seguintes classificadores:

* Naive Bayes
* Classificador Quadrático (utilizando regularização de friedman e matriz de covariância agregada)
* Classificador Quadrático Gaussiano (utilizando regularização de friedman e matriz de covariância agregada)
* Distância Mínima ao Centróide
* K Vizinhos Mais Próximos (usando k=11)
* Vizinho Mais Próximo

In [5]:
def run_experiments(data, times = 100):
    logger.setLevel(logging.CRITICAL)
    models = {
        "NB": NormalNaiveBayes(),
        "CQ(P)": 
            QuadraticClassifier(check_invertibility=True,pinv_mode="pooled"),
        "CQG(P)": 
            QuadraticGaussianClassifier(check_invertibility=True,pinv_mode="pooled"),
        "CQ(F)": 
            QuadraticClassifier(check_invertibility=True,pinv_mode="friedman"),
        "CQG(F)": 
            QuadraticGaussianClassifier(check_invertibility=True,pinv_mode="friedman"),
        "DMC": NearestCentroidClassifier(),
        "KNN(k=11)": KNeighborsClassifier(n_neighbors=11),
        "NN": KNeighborsClassifier(n_neighbors=1)
    }

    acc_results = []
    metrics_results = []
    for model_name in models:
        model = models[model_name]
        min_score = 101
        max_score = -1

        scores = []
        sensitivities = []
        specificities = []
        precisions = []
        for _ in range(times):
            
            train,test = train_test_split(data,.8, shuffle=True)
            model.fit(train[:,:-1],train[:,-1])
            
            predicted = []
            for x in test[:,:-1]:
                try:
                    y = model.predict(x)
                except Exception:
                    y = model.predict(x.reshape(1, -1))
                    
                if isinstance(y,list):
                    predicted.append(y[0])
                else:
                    predicted.append(y)
                
            conf_matrix = ConfusionMatrix(test[:,-1],predicted)

            sensitivities.append(conf_matrix.sensitivity())
            specificities.append(conf_matrix.specificity())
            precisions.append(conf_matrix.precision())

            score = conf_matrix.accuracy()
            scores.append(score)

            if score > max_score:
                max_score = score

            if score < min_score:
                min_score = score

        mean = np.mean(scores)
        std = np.std(scores)
        median = np.median(scores)

        acc_results.append({
            "1 - Alg": model_name,
            "2 - Média(%)": mean*100,
            "3 - Mediana(%)": median*100,
            "4 - Min/Max(%)": "{:.1f} / {:.1f}".format(min_score*100,max_score*100),
            "5 - Desv. Pad.(%)": std*100
        })
        
        metrics_results.append({
            "1 - Alg": model_name,
            "6 - Sensibilidade(%)": np.mean(sensitivities)*100,
            "7 - Especificidade(%)": np.mean(specificities)*100,
            "8 - Precisão(%)": np.mean(precisions)*100
        })

    return pd.DataFrame(acc_results).round(1), pd.DataFrame(metrics_results).round(1)

Executando os experimentos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [6]:
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,70.6,71.6,60.3 / 76.7,4.3
1,CQ(P),66.2,77.7,21.2 / 79.0,22.6
2,CQG(P),63.9,77.7,20.9 / 79.0,24.2
3,CQ(F),77.9,77.9,76.9 / 79.3,0.5
4,CQG(F),63.5,77.7,21.0 / 79.3,24.4
5,DMC,53.6,53.6,51.8 / 55.1,0.6
6,KNN(k=11),76.8,76.8,75.5 / 78.3,0.5
7,NN,68.7,68.8,67.5 / 70.0,0.6


Obtemos, também, as seguintes métricas:

In [7]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,65.3,72.1,40.6
1,CQ(P),21.0,79.0,4.7
2,CQG(P),25.0,75.0,5.5
3,CQ(F),0.0,100.0,0.0
4,CQG(F),26.0,74.0,5.8
5,DMC,66.8,49.9,27.4
6,KNN(k=11),12.5,95.1,41.8
7,NN,28.8,80.1,29.1


## Gerando Protótipos
---

A fim de diminuir a quantidade de dados para o treinamento, podemos utilizar o algoritmo *K-Médias* para gerar protótipos sobre os dados de cada classe.

Como os centróides no algoritmo são inicializados randomicamente, podemos aplicar o algoritmo uma certa quantidade de vezes(definido pela variável *attempts*) e obter o conjunto de centróides com menor *soma das distâncias quadráticas*.

In [9]:
from sklearn.cluster import KMeans

def generate_prototypes(data,k,attempts = 10):
    classes = np.unique(data[:,-1])
    n = data.shape[1]
    clusterized_data = np.reshape(np.zeros(n),(1,n))
    for c in classes:
        prototypes = []
        d = data[data[:,-1]==c]
        for _ in range(attempts):
            model = KMeans(n_clusters=k, max_iter = 10)
            y = model.fit_predict(d[:,:-1])
            ssd = 0
            for x,y in zip(d,y):
                ssd += np.linalg.norm(model.cluster_centers_[y] - x[:-1])
            
            prototypes.append((ssd,model.cluster_centers_))
        
        best_prototype = sorted(prototypes, key= lambda x: x[0])[0][1]
        
        y = np.array([c for _ in range(k)])
        clusterized_data = np.append(clusterized_data,np.c_[best_prototype,y],axis=0)

    return clusterized_data[1:,:]

Vamos gerar protótipos para as classes utilizando as quantidades 1000, 2000 e 3000 e comparar seus resultados.

Para fins de praticidade, serializamos os dados para não precisar recomputá-los em outros momentos

In [10]:
for k in [1000,2000,3000]:
    data = np.loadtxt("../datasets/default of credit card clients.csv", delimiter=',',skiprows=1)
    print("Generating {} prototypes".format(k))
    data = generate_prototypes(data, k)
    np.savetxt("clustered-data-{}.csv".format(k),data,delimiter=',')

Generating 1000 prototypes
Generating 2000 prototypes
Generating 3000 prototypes


Executando os experimentos para 1000 protótipos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [11]:
data = np.loadtxt("clustered-data-1000.csv",delimiter=',')
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,68.9,68.8,63.5 / 75.2,2.1
1,CQ(P),50.4,50.2,44.5 / 57.8,2.6
2,CQG(P),49.8,49.8,44.8 / 55.0,2.0
3,CQ(F),50.0,50.0,42.8 / 54.5,1.9
4,CQG(F),49.8,50.0,44.8 / 55.5,2.4
5,DMC,61.0,61.0,55.8 / 66.0,2.1
6,KNN(k=11),60.6,60.8,53.0 / 65.8,2.1
7,NN,36.4,36.2,32.2 / 41.2,2.0


Obtemos, também, as seguintes métricas:

In [12]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,88.0,49.7,63.9
1,CQ(P),55.0,45.0,27.7
2,CQG(P),46.0,54.0,23.0
3,CQ(F),0.0,100.0,0.0
4,CQG(F),53.0,47.0,26.4
5,DMC,66.3,55.7,59.8
6,KNN(k=11),71.4,49.9,58.9
7,NN,32.8,40.1,35.3


Executando os experimentos para 2000 protótipos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [13]:
data = np.loadtxt("clustered-data-2000.csv",delimiter=',')
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,70.9,71.0,67.6 / 73.5,1.4
1,CQ(P),50.3,50.2,46.9 / 54.5,1.7
2,CQG(P),49.9,50.1,45.4 / 53.1,1.5
3,CQ(F),49.8,49.9,45.8 / 53.6,1.6
4,CQG(F),49.9,50.1,44.8 / 53.8,1.8
5,DMC,64.7,64.6,60.6 / 69.0,1.9
6,KNN(k=11),69.6,69.6,65.4 / 71.9,1.3
7,NN,45.7,45.8,41.1 / 49.0,1.6


Obtemos, também, as seguintes métricas:

In [14]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,89.2,52.7,65.3
1,CQ(P),51.0,49.0,25.5
2,CQG(P),41.0,59.0,20.4
3,CQ(F),0.0,100.0,0.0
4,CQG(F),54.0,46.0,26.9
5,DMC,71.2,58.2,62.8
6,KNN(k=11),80.1,59.0,66.2
7,NN,41.8,49.6,45.2


Executando os experimentos para 3000 protótipos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [15]:
data = np.loadtxt("clustered-data-3000.csv",delimiter=',')
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,71.7,71.8,68.2 / 75.4,1.3
1,CQ(P),50.0,49.9,46.5 / 53.0,1.4
2,CQG(P),50.0,49.8,47.5 / 52.9,1.3
3,CQ(F),49.8,49.8,46.9 / 52.9,1.3
4,CQG(F),50.0,50.3,46.3 / 53.2,1.4
5,DMC,66.4,66.4,63.3 / 70.2,1.3
6,KNN(k=11),73.4,73.5,71.1 / 76.8,1.1
7,NN,52.8,52.9,49.4 / 55.5,1.3


Obtemos, também, as seguintes métricas:

In [16]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,90.5,52.8,65.7
1,CQ(P),47.0,53.0,23.4
2,CQG(P),47.0,53.0,23.4
3,CQ(F),0.0,100.0,0.0
4,CQG(F),51.0,49.0,25.6
5,DMC,74.1,58.6,64.4
6,KNN(k=11),81.9,65.0,69.8
7,NN,49.4,56.3,53.0


## LDA
---

Uma outra abordagem para otimizar os recursos computacionais é realizar a *Análise do Discriminante Linear*(LDA).

In [17]:
data = np.loadtxt("../datasets/default of credit card clients.csv", delimiter=',',skiprows=1)

def statistical_normalization(X):
    m = np.mean(X, axis = 0)
    std = np.std(X, axis = 0)
    normalized_X = np.zeros((X.shape[0],X.shape[1]))
    for i,x in enumerate(X):
        normalized_X[i] = (x - m)/std
        
    return normalized_X

data[:,:-1] = statistical_normalization(data[:,:-1])

In [18]:
classes = {}
for (x,y) in zip(data[:,:-1],data[:,-1]):
    if not y in classes:
        classes[y] = []

    classes[y].append(x)

classes = {k: np.array(classes[k]) for k in classes}

n = data.shape[1]-1

sw = np.zeros((n,n))
m = np.mean(data[:,:-1],axis=0)
sb = np.zeros((n,n))

for k in classes:
    ni = classes[k].shape[0]
    mi = np.mean(classes[k],axis=0).reshape(n,1)
    mm = mi-m
#     sb += ni*np.outer(mi-m,mi-m)
    sb += ni * mm.dot(mm.T)
    
    s = np.zeros((n,n))
    for x in classes[k]:
        z = (x.reshape(n,1)-mi)
#         s += np.outer(z,z)
        s += z.dot(z.T)
    sw += s
    
Z = np.linalg.inv(sw).dot(sb)
values, vectors = np.linalg.eig(Z,)

values = np.real(values)
vectors = np.real(vectors)

idx = values.argsort()[::-1]   
values = values[idx]
vectors = vectors[:,idx]

values

array([ 3.39780897e+00,  1.64947835e-15,  4.69163593e-16,  4.69163593e-16,
        9.43119860e-17,  5.20827676e-17,  4.21570204e-17,  4.21570204e-17,
        1.57600076e-17,  4.35527421e-19,  0.00000000e+00, -1.44799293e-18,
       -1.44799293e-18, -6.61059612e-18, -2.22001562e-17, -2.22001562e-17,
       -5.15544612e-17, -6.96727862e-17, -6.96727862e-17, -1.54450465e-16,
       -4.57577266e-16, -4.57577266e-16, -1.34804604e-15, -2.98158464e-15])

Como possuímos apenas um apenas um autovalor de magnitude relevante, vamos reduzir a dimensão dos dados para 1.

In [19]:
pca_vectors = vectors[:,0]

data = np.c_[np.matmul(data[:,:-1],pca_vectors), data[:,-1]]
data.shape

(30000, 2)

Executando os experimentos sobre os novos dados, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [20]:
acc_df, metrics_df = run_experiments(data)
acc_df

,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,81.4,81.4,80.3 / 82.5,0.4
1,CQ(P),73.9,73.9,72.6 / 74.8,0.5
2,CQG(P),77.9,77.9,76.7 / 79.4,0.5
3,CQ(F),73.8,73.8,72.5 / 74.9,0.5
4,CQG(F),77.8,77.8,76.6 / 79.0,0.5
5,DMC,72.6,72.5,71.1 / 74.1,0.5
6,KNN(k=11),80.7,80.7,79.6 / 81.9,0.4
7,NN,71.6,71.6,70.5 / 72.8,0.5


Obtemos, também, as seguintes métricas:

In [21]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,29.6,96.1,68.6
1,CQ(P),58.1,78.3,43.2
2,CQG(P),0.0,100.0,0.0
3,CQ(F),58.2,78.2,43.1
4,CQG(F),0.0,100.0,0.0
5,DMC,60.1,76.1,41.6
6,KNN(k=11),33.3,94.2,61.8
7,NN,35.8,81.8,36.0


Sobre os dados gerados após o LDA, podemos aplicar a geração dos protótipos novamente.

In [22]:
for k in [1000,2000,3000]:
    data = np.loadtxt("../datasets/default of credit card clients.csv", delimiter=',',skiprows=1)
    print("Generating {} LDA prototypes".format(k))
    data = generate_prototypes(data, k)
    np.savetxt("lda-clustered-data-{}.csv".format(k),data,delimiter=',')

Generating 1000 LDA prototypes
Generating 2000 LDA prototypes
Generating 3000 LDA prototypes


Executando os experimentos para 1000 protótipos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [23]:
data = np.loadtxt("lda-clustered-data-1000.csv",delimiter=',')
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,68.5,68.5,64.0 / 73.8,1.9
1,CQ(P),49.8,50.2,42.5 / 54.5,2.3
2,CQG(P),49.8,49.9,44.8 / 55.0,2.2
3,CQ(F),50.0,49.9,45.2 / 56.2,2.3
4,CQG(F),49.8,49.8,45.2 / 54.2,2.1
5,DMC,60.9,61.3,56.0 / 66.8,2.3
6,KNN(k=11),60.6,60.8,54.0 / 65.0,2.0
7,NN,36.1,36.1,30.2 / 42.0,2.3


Obtemos, também, as seguintes métricas:

In [24]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,88.3,49.0,63.2
1,CQ(P),55.0,45.0,27.4
2,CQG(P),50.0,50.0,24.9
3,CQ(F),0.0,100.0,0.0
4,CQG(F),55.0,45.0,27.3
5,DMC,67.4,54.3,59.9
6,KNN(k=11),71.7,49.7,58.8
7,NN,32.3,40.0,35.3


Executando os experimentos para 2000 protótipos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [25]:
data = np.loadtxt("lda-clustered-data-2000.csv",delimiter=',')
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,71.0,71.0,68.1 / 75.8,1.5
1,CQ(P),50.0,49.9,45.9 / 54.0,1.5
2,CQG(P),50.1,50.0,46.1 / 55.1,1.6
3,CQ(F),50.3,50.4,45.6 / 53.5,1.6
4,CQG(F),50.0,50.0,45.9 / 54.2,1.5
5,DMC,64.4,64.3,60.8 / 68.2,1.4
6,KNN(k=11),69.4,69.4,65.9 / 72.5,1.3
7,NN,45.9,46.0,42.4 / 50.5,1.5


Obtemos, também, as seguintes métricas:

In [26]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,89.5,52.4,65.4
1,CQ(P),44.0,56.0,22.0
2,CQG(P),45.0,55.0,22.5
3,CQ(F),0.0,100.0,0.0
4,CQG(F),48.0,52.0,23.8
5,DMC,71.1,57.7,62.7
6,KNN(k=11),79.5,59.4,66.1
7,NN,41.8,50.0,45.3


Executando os experimentos para 3000 protótipos, obtemos as seguintes informações sobre as acurácias dos algoritmos:

In [27]:
data = np.loadtxt("lda-clustered-data-3000.csv",delimiter=',')
acc_df, metrics_df = run_experiments(data)
acc_df

/home/lucaspg/miniconda3/envs/libra/lib/python3.6/site-packages/numpy/linalg/linalg.py:1874: RuntimeWarning: overflow encountered in det
  r = _umath_linalg.det(a, signature=signature)


,1 - Alg,2 - Média(%),3 - Mediana(%),4 - Min/Max(%),5 - Desv. Pad.(%)
0,NB,71.7,71.8,67.8 / 74.4,1.2
1,CQ(P),50.0,50.2,46.6 / 53.4,1.4
2,CQG(P),50.0,49.8,46.7 / 54.3,1.2
3,CQ(F),50.2,50.1,47.3 / 54.1,1.2
4,CQG(F),50.1,49.9,46.8 / 53.2,1.2
5,DMC,66.2,66.2,63.2 / 69.8,1.3
6,KNN(k=11),73.6,73.6,70.8 / 76.6,1.1
7,NN,53.0,53.1,49.3 / 56.1,1.3


Obtemos, também, as seguintes métricas:

In [29]:
metrics_df

,1 - Alg,6 - Sensibilidade(%),7 - Especificidade(%),8 - Precisão(%)
0,NB,90.4,52.9,65.8
1,CQ(P),49.0,51.0,24.5
2,CQG(P),49.0,51.0,24.5
3,CQ(F),0.0,100.0,0.0
4,CQG(F),42.0,58.0,21.1
5,DMC,74.1,58.4,64.0
6,KNN(k=11),82.3,64.8,70.0
7,NN,49.6,56.4,52.9


## Conclusão
---

